In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import re
import json
import webbrowser
import pandas as pd
import ipywidgets as widgets
from datetime import datetime
from ipywidgets import AppLayout, Button, Layout
from ipywidgets import Button, Layout, jslink, IntText, IntSlider
from IPython.display import display, Javascript, Markdown, clear_output, HTML
from ipywidgets import GridspecLayout
from markdown_it import MarkdownIt

In [2]:
# Creates navigation file when first visit this notebook
# Define the content of the JSON file
json_content = {
    "obj_list": [["Select a notebook", None]],
    "st_area_list": [["Select a notebook", None]],
    "dt_list": [["Select a notebook", None]],
    "ts_list": [["Select a notebook", None]],
    "ed_list": [["Select a notebook", None]],
    "da_list": [["Select a notebook", None]],
    "rec_list": [["Select a notebook", None]]
}

# Define the file path
file_path = "../data_files/json/ques_nav_list.json"

# Ensure the directory exists
os.makedirs(os.path.dirname(file_path), exist_ok=True)

# Write the JSON content to the file
with open(file_path, "w") as json_file:
    json.dump(json_content, json_file, indent=2)


In [3]:
%run ../02_ui_components/progressbar.ipynb

In [4]:
%run ../02_ui_components/sidebar.ipynb

In [5]:
%run ../02_ui_components/dropdown.ipynb

In [6]:
%run ../02_ui_components/header_button.ipynb

In [7]:
%run ../02_ui_components/more_info.ipynb

In [8]:
def get_css_class(css_file, class_name):
    with open(css_file, 'r') as file:
        css = file.read()
        # Use regex to extract the class
        pattern = r'\.' + re.escape(class_name) + r'\s*\{([^}]*)\}'
        match = re.search(pattern, css, re.DOTALL)
        if match:
            css_content = match.group(1).strip()
            return f'\n<style>\n.{class_name} {{\n{css_content}\n}}\n</style>\n'
        else:
            return None

In [9]:
css_path = "../css/style.css"

In [10]:
# Custom CSS to modify the question button design
ques_widget_css = get_css_class(css_path, 'ques-widget')

In [12]:
# Custom CSS to modify the question button design
ques_button_css = get_css_class(css_path, 'ques-button')

In [13]:
# Custom CSS to modify the option button design
opt_button_css = get_css_class(css_path, 'opt-button')

In [14]:
# Custom CSS to modify the header design
header_css = get_css_class(css_path, 'custom-header')

In [15]:
# Custom CSS to modify the footer design
footer_css = get_css_class(css_path, 'custom-footer')

In [16]:
# Create an HTML widget for the background color
background_color_html = widgets.HTML(
    value = get_css_class(css_path, 'custom-vbox')
)

In [17]:
# # Loading the header image within an image widget
# header_prog1 = widgets.Image(value=open("../03_icons/icon_level-banner_1-0_objective-resources.png", "rb").read(), format='png',
#                       layout=widgets.Layout(width='auto', height='70px'))

# header_prog2 = widgets.Image(value=open("../03_icons/icon_level-banner_2-1_project-level_study-area_site-selection.png", "rb").read(), format='png',
#                       layout=widgets.Layout(width='auto', height='70px'))

# header_prog3 = widgets.Image(value=open("../03_icons/icon_level-banner_2-2_project-level_duration-timing.png", "rb").read(), format='png',
#                       layout=widgets.Layout(width='auto', height='70px'))

# header_prog4 = widgets.Image(value=open("../03_icons/icon_level-banner_4-0_target-species.png", "rb").read(), format='png',
#                       layout=widgets.Layout(width='auto', height='70px'))

# header_prog5 = widgets.Image(value=open("../03_icons/icon_level-banner_5-1_equipment-deployment.png", "rb").read(), format='png',
#                       layout=widgets.Layout(width='auto', height='70px'))

# header_prog6 = widgets.Image(value=open("../03_icons/icon_level-banner_6-0_data-analysis.png", "rb").read(), format='png',
#                       layout=widgets.Layout(width='auto', height='70px'))

In [18]:
# Taking from separate header notebook
header = header_prog1

In [19]:
# Creating a container for header button
header_container = widgets.VBox([header, header_widget], 
                    layout=widgets.Layout(width='95%', padding_top= '10px', align_items='flex-start', justify_content='center'))

In [20]:
# Creating button
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='auto', width='auto', border='1px solid black')) #width='max-content'

In [21]:
# Declaring necessary variables and importing neccesary files for storing user answers and keep track of the visited pages
global stored_choices, path

stored_choices = {}
path = "../data_files/json/temp_storage.json"
back_path = "../data_files/json/backtracking.json"
dropdown_path = "../data_files/csv/links.csv"
ques_nav_path = "../data_files/json/ques_nav_list.json"
question_path = "../data_files/csv/question_data.csv"
button_color_path = "../data_files/csv/button_color.csv"

In [22]:
# Importing the json file for storing user answers
with open(path, "r") as f:
    stored_choices = json.load(f)

In [23]:
# Importing the json file for keeping track of visited pages
with open(back_path, 'r') as f:
    back_list = json.load(f)

In [24]:
# Reading the CSV data file into a DataFrame
df = pd.read_csv(question_path, low_memory=False)

In [25]:
dp_df = pd.read_csv(dropdown_path, low_memory=False)

In [26]:
btn_color = pd.read_csv(button_color_path, low_memory=False)

In [27]:
drop_link = dp_df.loc[dp_df['field_code'] == 'objective', 'info_url'].iloc[0]

In [28]:
notebook_name = "01_02_objective"
page_dropdown_text = df.loc[df['page_name'] == 'objective', 'page_dropdown_text'].iloc[0]
notebook_list_name = "obj_list"
footer_name = "footer_in_prog1"

In [29]:
footer = main(notebook_name, notebook_list_name, footer_name, page_dropdown_text)

In [30]:
# Assigns dynamic link to the last sidebar button based on the notebook dropdown info link
concept_grid[5, 0:9].on_click(on_concept_option_clicked(drop_link))

In [31]:
# seperates the string based on ',' also removes the whitespace after comma & makes a list
def split_and_trim(s):
    return [item.strip() for item in s.split(',')]

In [32]:
# gets the question based on value of other column
ques = df.loc[df['question_code'] == 'objective', 'question_text'].iloc[0]
opt = df.loc[df['question_code'] == 'objective', 'q_option_label'].iloc[0]
opt_codes = df.loc[df['question_code'] == 'objective', 'q_option_code'].iloc[0]

# Using the split_and_trim function for neccessary steps
opt = split_and_trim(opt)
opt_codes = split_and_trim(opt_codes)

In [33]:
# html_ques = f"""
#     <div>
#         <p>{ques}</p>
#     </div>
#     """
# ques = widgets.HTML(value=html_ques)

In [34]:
# widget on right side for more information
html_widget = more_info_widget("../data_files/markdown/01_02_objective_note.md")

In [36]:
# Factor widget
question = f"""
    <div class="ques-widget">
        <p>{ques}</p>
    </div>
    """
question = widgets.HTML(value=question, layout=Layout(max_width='700px', padding='5px'))

# Add custom class to the button
# question.add_class(ques_widget_css) # Adding custom css properties to the buttons

# question = create_expanded_button(ques, 'success')
# question.style.button_color = 'lightblue'
# question.style.text_color = 'black'

# # Add custom class to the button
# question.add_class('ques-button') # Adding custom css properties to the buttons

option1 = create_expanded_button(opt[0], 'success')
option1.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option1.style.text_color = 'black'
option1.add_class('opt-button')

option2 = create_expanded_button(opt[1], 'success')
option2.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option2.style.text_color = 'black'
option2.add_class('opt-button')

option3 = create_expanded_button(opt[2], 'success')
option3.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option3.style.text_color = 'black'
option3.add_class('opt-button')

option4 = create_expanded_button(opt[3], 'success')
option4.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option4.style.text_color = 'black'
option4.add_class('opt-button')

option5 = create_expanded_button(opt[4], 'success')
option5.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option5.style.text_color = 'red'
option5.add_class('opt-button')

option6 = create_expanded_button(opt[5], 'success')
option6.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option6.style.text_color = 'black'
option6.add_class('opt-button')

option7 = create_expanded_button(opt[6], 'success')
option7.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option7.style.text_color = 'black'
option7.add_class('opt-button')

option8 = create_expanded_button(opt[7], 'success')
option8.style.button_color = btn_color.loc[btn_color['Category'] == 'Regular Button', 'Color_Code'].iloc[0]
option8.style.text_color = 'black'
option8.add_class('opt-button')

option9 = create_expanded_button(opt[8], 'success')
option9.style.button_color = btn_color.loc[btn_color['Category'] == 'Unknown Button', 'Color_Code'].iloc[0]
option9.style.text_color = 'black'
option9.add_class('opt-button')

# Create a HTML widget to include the custom CSS
css_widget = widgets.HTML(value=ques_widget_css)
css_widget2 = widgets.HTML(value=opt_button_css)

factor_grid = GridspecLayout(10, 18, grid_gap='5px', height='550px')
factor_grid[4:8, 1:6] = question
factor_grid[1:8, 11:18] = html_widget
factor_grid[1, 7:10] = option1
factor_grid[2, 7:10] = option2
factor_grid[3, 7:10] = option3
factor_grid[4, 7:10] = option4
factor_grid[5, 7:10] = option5
factor_grid[6, 7:10] = option6
factor_grid[7, 7:10] = option7
factor_grid[8, 7:10] = option8
factor_grid[9, 7:10] = option9
# factor_grid[10, 7:9] = option10

# Creating a container to store the factor widget and added custom css widgets
logical_integer_container = widgets.VBox(children=[factor_grid, css_widget, css_widget2], layout=widgets.Layout(
    align_items='center'  # Center align vertically
#     border='solid 2px black',
#     width = "70%"
#     justify_content='center'  # Center align horizontally
#     height='100vh'  # Set height to 100% of viewport height for vertical centering
))

Button(button_style='success', description='What is the variable of interest?<br>Select "Unknown" if you\'re n…

In [32]:
# Creating button list for implementing the checkmark feature
button_list = [option1, option2, option3, option4, option5, option6, option7, option8, option9]

In [33]:
selected = False

In [34]:
# Create the text widget with a warning message
warning_widget2 = widgets.HTML(
    value="""
    <div style="padding: 5px; background-color: #f8d7da; color: #721c24; border: 2px solid #f5c6cb; border-radius: 5px;">
        <strong>Warning!</strong> Unavailable for now. Please select another.
    </div>
    """
)

In [35]:
error2 = widgets.HBox(
    [warning_widget2],
    layout=widgets.Layout(
        width='auto',  
        height='auto',
        margin='auto',  
        align_items='flex-end',  
        justify_content='flex-end' 
    )
)

In [36]:
selected_option_list = []

In [37]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_1_button_clicked(option1):
    with output:
        clear_output()
        global selected
        
        selected = True
        
        # storing all the selected option information
        selected_option = factor_grid[1, 7:10].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option1:
                if btn.icon == 'check':
                    btn.icon=''
                    selected_option_list.remove(selected_option_code)
                else:
                    btn.icon='check'
                    if selected_option_code not in selected_option_list:    
                        selected_option_list.append(selected_option_code)        
        
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_list
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
        
# Calling the on click function
factor_grid[1, 7:10].on_click(option_1_button_clicked)

In [38]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_2_button_clicked(option2):
    with output:
        clear_output()
        global selected
        
        selected = True
        
        # storing all the selected option information
        selected_option = factor_grid[2, 7:10].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option2:
                if btn.icon == 'check':
                    btn.icon=''
                    selected_option_list.remove(selected_option_code)
                else:
                    btn.icon='check'
                    if selected_option_code not in selected_option_list:    
                        selected_option_list.append(selected_option_code)
        
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_list
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function        
factor_grid[2, 7:10].on_click(option_2_button_clicked)

In [39]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_3_button_clicked(option3):
    with output:
        clear_output()
        global selected
        
        selected = True
        
        # storing all the selected option information
        selected_option = factor_grid[3, 7:10].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option3:
                if btn.icon == 'check':
                    btn.icon=''
                    selected_option_list.remove(selected_option_code)
                else:
                    btn.icon='check'
                    if selected_option_code not in selected_option_list:    
                        selected_option_list.append(selected_option_code)
        
        
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_list
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function       
factor_grid[3, 7:10].on_click(option_3_button_clicked)

In [40]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_4_button_clicked(option4):
    with output:
        clear_output()
        global selected
        
        selected = True
        
        # storing all the selected option information
        selected_option = factor_grid[4, 7:10].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option4:
                if btn.icon == 'check':
                    btn.icon=''
                    selected_option_list.remove(selected_option_code)
                else:
                    btn.icon='check'
                    if selected_option_code not in selected_option_list:    
                        selected_option_list.append(selected_option_code)
        
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_list
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function       
factor_grid[4, 7:10].on_click(option_4_button_clicked)

In [41]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_5_button_clicked(option5):
    with output:
        clear_output()
        global selected
        
        selected = False
        
#       storing all the selected option information
#       selected_option = factor_grid[5, 7:10].description
#       selected_option_index = opt.index(selected_option)
#       selected_option_code = opt_codes[selected_option_index]

        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option5:
                if btn.icon == 'check':
                    btn.icon=''
#                     selected_option_list.remove(selected_option_code)
                else:
                    btn.icon='check'
#                     if selected_option_code not in selected_option_list:    
#                         selected_option_list.append(selected_option_code)
        display(error2)
        
#         # Retrieving infomation for using inside logic while determining the next page
#         stored_choices["objective"] = selected_option_code
        
#         # Storing all the information
#         with open(path, "w") as f:
#             json.dump(stored_choices, f)
            
# Calling the on click function       
factor_grid[5, 7:10].on_click(option_5_button_clicked)

In [42]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_6_button_clicked(option6):
    with output:
        clear_output()
        global selected
        
        selected = False
        
#         # storing all the selected option information
#         selected_option = factor_grid[6, 7:10].description
#         selected_option_index = opt.index(selected_option)
#         selected_option_code = opt_codes[selected_option_index]
        
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option6:
                if btn.icon == 'check':
                    btn.icon=''
#                     selected_option_list.remove(selected_option_code)
                else:
                    btn.icon='check'
#                     if selected_option_code not in selected_option_list:    
#                         selected_option_list.append(selected_option_code)
        display(error2)
        
        
#         # Retrieving infomation for using inside logic while determining the next page
#         stored_choices["objective"] = selected_option_code
        
#         # Storing all the information
#         with open(path, "w") as f:
#             json.dump(stored_choices, f)
            
# Calling the on click function     
factor_grid[6, 7:10].on_click(option_6_button_clicked)

In [43]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_7_button_clicked(option7):
    with output:
        clear_output()
        global selected
        
        selected = False
        
#         # storing all the selected option information
#         selected_option = factor_grid[7, 7:10].description
#         selected_option_index = opt.index(selected_option)
#         selected_option_code = opt_codes[selected_option_index]
        
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option7:
                if btn.icon == 'check':
                    btn.icon=''
#                     selected_option_list.remove(selected_option_code)
                else:
                    btn.icon='check'
#                     if selected_option_code not in selected_option_list:    
#                         selected_option_list.append(selected_option_code)
                
        display(error2)       
                
        
#         # Retrieving infomation for using inside logic while determining the next page
#         stored_choices["objective"] = selected_option_code
        
#         # Storing all the information
#         with open(path, "w") as f:
#             json.dump(stored_choices, f)



            
# Calling the on click function     
factor_grid[7, 7:10].on_click(option_7_button_clicked)

In [44]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_8_button_clicked(option8):
    with output:
        clear_output()
        global selected
        
        selected = True
        
        # storing all the selected option information
        selected_option = factor_grid[8, 7:10].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option8:
                if btn.icon == 'check':
                    btn.icon=''
                    selected_option_list.remove(selected_option_code)
                else:
                    btn.icon='check'
                    if selected_option_code not in selected_option_list:    
                        selected_option_list.append(selected_option_code)
        
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_code
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function       
factor_grid[8, 7:10].on_click(option_8_button_clicked)

In [45]:
# Implementing the steps when a button is clicked
output = widgets.Output()
def option_9_button_clicked(option9):
    with output:
        clear_output()
        global selected
        
        selected = True
        
        # storing all the selected option information
        selected_option = factor_grid[9, 7:10].description
        selected_option_index = opt.index(selected_option)
        selected_option_code = opt_codes[selected_option_index]
        
        # changes the button icon to a checkmark when clicked
        for btn in button_list:
            if btn == option9:
                if btn.icon == 'check':
                    btn.icon=''
                    selected_option_list.remove(selected_option_code)
                else:
                    btn.icon='check'
                    if selected_option_code not in selected_option_list:    
                        selected_option_list.append(selected_option_code)
        
        
        # Retrieving infomation for using inside logic while determining the next page
        stored_choices["objective"] = selected_option_code
        
        # Storing all the information
        with open(path, "w") as f:
            json.dump(stored_choices, f)
            
# Calling the on click function      
factor_grid[9, 7:10].on_click(option_9_button_clicked)

In [46]:
# # button.on_click(show_popup(None, drop_link))
# button.on_click(lambda b: show_popup(b, drop_link))

In [47]:
# close_button.on_click(close_popup)

In [ ]:
accordion.observe(lambda change: show_popup(accordion, drop_link), names='selected_index')

In [47]:
# Create a buttons for submitting the question
submit_button = widgets.Button(description="Submit & Next >>", layout=widgets.Layout(border='1px solid black'))

In [48]:
# Create a button for going back to previous question
back_button = widgets.Button(description="<< Back", layout=widgets.Layout(border='1px solid black'))

In [49]:
# Create the text widget with a warning message
warning_widget = widgets.HTML(
    value="""
    <div style="padding: 5px; background-color: #f8d7da; color: #721c24; border: 2px solid #f5c6cb; border-radius: 5px;">
        <strong>Warning!</strong> Select an option first.
    </div>
    """
)

In [50]:
error = widgets.HBox(
    [warning_widget],
    layout=widgets.Layout(
        width='auto',  
        height='auto',
        margin='auto',  
        align_items='flex-end',  
        justify_content='flex-end' 
    )
)

In [51]:
# For going back to last question

output = widgets.Output()
def on_back_button_clicked(event):
    with output:
        clear_output()
        file_name = back_list[-1]
        notebook_filename = file_name + ".ipynb"
        js_code = f"window.open('{notebook_filename}', '_self');"
#         link = Markdown('''<a href="{}" target="_self" rel="noopener noreferrer">Previous</a>'''.format(notebook_filename))
        back_list.pop()
        with open (back_path, "w")as f:
            json.dump(back_list, f)
        display(Javascript(js_code))

back_button.on_click(on_back_button_clicked)

In [52]:
output = widgets.Output()
def on_button_clicked(event):
    with output:
#         clear_output()

        if not selected:
            display(error)
        else:
            next_notebook = "01_03_num_cams"

            notebook_filename = next_notebook + ".ipynb"
            # JavaScript to open the URL
            js_code = f"window.open('{notebook_filename}', '_self');"

            # Storing current page name for later use when backtracking
            back_list.append("01_02_objective")
            with open (back_path, "w")as f:
                json.dump(back_list, f)

            display(Javascript(js_code))

submit_button.on_click(on_button_clicked)

In [53]:
# Creating containers for all the widgets along with their css properties

all_button = widgets.HBox(
    [back_button, submit_button],
    layout=widgets.Layout(
        width='100%',  
        height='auto',
#         border='solid 2px grey',
        padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='space-between' 
    )
)
buttons = widgets.VBox(
    [output, all_button],
    layout=widgets.Layout(
        width='100%',  
        height='auto',  
#         border='solid 2px grey',
        padding='1px',
        margin='auto',  
        align_items='center',  
        justify_content='center'  
    )
)



upper = widgets.VBox(
    [logical_integer_container, buttons],
    layout=widgets.Layout(
        width='95%',  
        height='auto', 
        border='solid 1px grey',
        padding='10px',
        margin='auto',  
        align_items='center',  
        justify_content='center' 
    )
)
upper.add_class("custom-vbox")

popup = widgets.VBox(children=[accordion], layout=widgets.Layout(
        width='95.3%',
        padding ='5px 0px 0px 0px'))

#  container2,
right_col = widgets.VBox(children=[header_container, background_color_html, upper, footer, popup], layout=widgets.Layout(
        width='88%',  #, popup
        height='100%',  
        border_left='solid 1px grey',
        padding='5px',
        margin='auto',  
        align_items='center',  
        justify_content='center'
))

cols = widgets.HBox(
    [left_col, right_col],
    layout=widgets.Layout(
        width='95%',  
        height='auto',  #90vh
        border='solid 2px grey',
#         padding='5px',
        margin='auto',  
        align_items='flex-start', 
        justify_content='center'
    )
)

display(cols)

In [15]:
# from IPython.display import display, HTML
# import ipywidgets as widgets

# # Custom CSS to modify the question button design
# acc_button_css = get_css_class(css_path, 'accordion-button')
# print(acc_button_css)

In [16]:
# # Create an Output widget
# output_popup = widgets.Output()
# with output_popup:
#     print("This is additional information inside the accordion.")

# # Create an Accordion widget
# accordion = widgets.Accordion(children=[output_popup], titles=('Click here for more information',))

# # Apply the custom CSS class
# accordion.add_class(acc_button_css)

# acc_widget = widgets.HTML(value=acc_button_css)
# # Display the Accordion
# # display(accordion)


In [17]:
# acc_container = widgets.VBox(children=[accordion, acc_widget])

In [18]:
# display(acc_container)